In [1]:
library(tidyverse)
library(blorr)
library(DescTools)
library(robustbase)
library(car)
library(stargazer)
library(haven)
library(mfx)

Warning message:
"package 'tidyverse' was built under R version 4.2.3"
Warning message:
"package 'ggplot2' was built under R version 4.2.3"
Warning message:
"package 'tibble' was built under R version 4.2.3"
Warning message:
"package 'tidyr' was built under R version 4.2.3"
Warning message:
"package 'readr' was built under R version 4.2.3"
Warning message:
"package 'purrr' was built under R version 4.2.3"
Warning message:
"package 'dplyr' was built under R version 4.2.3"
Warning message:
"package 'stringr' was built under R version 4.2.3"
Warning message:
"package 'forcats' was built under R version 4.2.3"
Warning message:
"package 'lubridate' was built under R version 4.2.3"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ──────────────────────────────────────

In [2]:
df <- read_dta("mus14data.dta")
head(df)

personid,private,eprhi,age,hisp,white,female,educyear,married,excel,⋯,agewhi,_est_blogit,_est_bprobit,_est_bOLS,_est_bloghet,_est_bprobhet,_est_bOLShet,_est_PROBIT,_est_IVPROBML,_est_IVPROB2S
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
54963010,0,0,62,0,0,1,12,0,0,⋯,0,1,1,1,1,1,1,1,1,1
21629010,0,0,59,0,1,1,12,0,0,⋯,59,1,1,1,1,1,1,1,1,1
20421010,0,0,60,0,0,0,13,0,0,⋯,0,1,1,1,1,1,1,1,1,1
38784010,0,0,62,0,1,1,10,0,0,⋯,62,1,1,1,1,1,1,1,1,1
55630030,0,0,54,0,1,1,9,0,0,⋯,54,1,1,1,1,1,1,1,1,1
15893020,0,0,62,0,1,1,12,1,0,⋯,62,1,1,1,1,1,1,1,1,1


In [3]:
df <- as.data.frame(df)

#### Select the variables to use

In [4]:
data <- df[c('ins', 'retire', 'age','hstatusg','hhincome','educyear','married','female','white','hisp','chronic','sretire','adl')]

In [5]:
head(data)

,ins,retire,age,hstatusg,hhincome,educyear,married,female,white,hisp,chronic,sretire,adl
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,0,62,0,0,12,0,1,0,0,3,0,0
2,0,0,59,0,0,12,0,1,1,0,1,0,3
3,0,1,60,1,0,13,0,0,0,0,2,0,0
4,0,0,62,0,0,10,0,1,1,0,4,0,3
5,0,0,54,0,0,9,0,1,1,0,6,0,0
6,0,1,62,1,0,12,1,1,1,0,0,1,0


In [6]:
data <- data %>% mutate(linc = log(1+data$hhincome))

In [7]:
stargazer(data,type='text')


Statistic   N    Mean  St. Dev.  Min     Max   
-----------------------------------------------
ins       3,206 0.387   0.487     0       1    
retire    3,206 0.625   0.484     0       1    
age       3,206 66.914  3.676    52      86    
hstatusg  3,206 0.705   0.456     0       1    
hhincome  3,206 45.264  64.339  0.000 1,312.124
educyear  3,206 11.899  3.305     0      17    
married   3,206 0.733   0.442     0       1    
female    3,206 0.478   0.500     0       1    
white     3,206 0.821   0.384     0       1    
hisp      3,206 0.073   0.260     0       1    
chronic   3,206 2.063   1.416     0       8    
sretire   3,206 0.388   0.487     0       1    
adl       3,206 0.302   0.825     0       5    
linc      3,206 3.427   0.889   0.000   7.180  
-----------------------------------------------


#### Run the Logit regression

In [8]:
lgmodel <- glm(ins ~ retire + age + hstatusg + hhincome + educyear + married + hisp,data=data,family=binomial(link="logit"))
summary(lgmodel)


Call:
glm(formula = ins ~ retire + age + hstatusg + hhincome + educyear + 
    married + hisp, family = binomial(link = "logit"), data = data)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-2.456  -1.009  -0.703   1.224   2.373  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.715578   0.748622  -2.292 0.021926 *  
retire       0.196930   0.084207   2.339 0.019354 *  
age         -0.014596   0.011287  -1.293 0.195969    
hstatusg     0.312265   0.091674   3.406 0.000659 ***
hhincome     0.002304   0.000762   3.023 0.002503 ** 
educyear     0.114263   0.014201   8.046 8.55e-16 ***
married      0.578636   0.093320   6.201 5.63e-10 ***
hisp        -0.810306   0.195751  -4.139 3.48e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 4279.5  on 3205  degrees of freedom
Residual deviance: 3989.8  on 3198  degrees of freedom
AIC: 4005.8

Nu

#### Get the results in odds ratio with mfx package

In [11]:
logitor(ins ~ retire + age + hstatusg + hhincome + educyear + married + hisp,data=data)

Call:
logitor(formula = ins ~ retire + age + hstatusg + hhincome + 
    educyear + married + hisp, data = data)

Odds Ratio:
         OddsRatio Std. Err.       z     P>|z|    
retire   1.2176584 0.1025350  2.3386 0.0193538 *  
age      0.9855105 0.0111235 -1.2931 0.1959695    
hstatusg 1.3665173 0.1252739  3.4063 0.0006586 ***
hhincome 1.0023063 0.0007638  3.0229 0.0025033 ** 
educyear 1.1210464 0.0159201  8.0460 8.553e-16 ***
married  1.7836040 0.1664454  6.2006 5.626e-10 ***
hisp     0.4447220 0.0870546 -4.1395 3.481e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

##### Interpretation

* The odds of ins = 1 are 1.12 times higher when educyear increases by 1 year (keeping other predictors constant)
* The odds of ins = 1 are 1.37 times higher when hhincome increases by $1
* The odds of ins = 1 are 1.78 times higher when marital status is married.

#### Get the model fit statistics using blorr package

In [12]:
lgmodelfit <- blr_model_fit_stats(lgmodel)
print(lgmodelfit)

                               Model Fit Statistics                                
----------------------------------------------------------------------------------
Log-Lik Intercept Only:     -2139.771    Log-Lik Full Model:             -1994.878 
Deviance(3198):              3989.757    LR(7):                            289.786 
                                         Prob > LR:                          0.000 
MCFadden's R2                   0.068    McFadden's Adj R2:                  0.064 
ML (Cox-Snell) R2:              0.086    Cragg-Uhler(Nagelkerke) R2:         0.117 
McKelvey & Zavoina's R2:        0.130    Efron's R2:                         0.082 
Count R2:                       0.624    Adj Count R2:                       0.030 
BIC:                         4054.339    AIC:                             4005.757 
----------------------------------------------------------------------------------



#### Comparison of binary models and parameter estimates

In [13]:
blogit <- glm(ins ~ retire + age + hstatusg + hhincome + educyear + married + hisp,data=data,family=binomial(link="logit"))
bprobit <- glm(ins ~ retire + age + hstatusg + hhincome + educyear + married + hisp,data=data,family=binomial(link="probit"))
blogitr <- glmrob(ins ~ retire + age + hstatusg + hhincome + educyear + married + hisp,
                  data=data,family=binomial(link="logit"),method="Mqle",control=glmrobMqle.control(tcc=1.2))
bprobitr <- glmrob(ins ~ retire + age + hstatusg + hhincome + educyear + married + hisp,
                  data=data,family=binomial(link="probit"),method="Mqle",control=glmrobMqle.control(tcc=1.2))

In [14]:
stargazer(blogit,bprobit,blogitr,bprobitr,title="Results",type="text", 
          column.labels=c("blogit","bprobit","blogitr","bprobitr"),align=TRUE)


Results
                             Dependent variable:           
                  -----------------------------------------
                                     ins                   
                   logistic    probit         robust       
                                                GLM        
                    blogit    bprobit    blogitr  bprobitr 
                     (1)        (2)        (3)       (4)   
-----------------------------------------------------------
retire             0.197**    0.118**    0.206**   0.126** 
                   (0.084)    (0.051)    (0.085)   (0.052) 
                                                           
age                 -0.015     -0.009    -0.015    -0.009  
                   (0.011)    (0.007)    (0.011)   (0.007) 
                                                           
hstatusg           0.312***   0.198***  0.255***  0.160*** 
                   (0.092)    (0.055)    (0.092)   (0.056) 
                               

### Hypothesis and Specification Tests

#### Specification Error Tests--Linktest
The idea behind linktest is that if the model is properly specified, one should not be able to find any additional predictors tha are statistically significant except by chance. The linktest uses the linear predicted value (fit) and liear predicted value square (fit2) as the predictors to rebuild the model.  It is expected that fit be statistically significant while fit2 should not.  If fit2 is significant, it signifies omitted variable problem.

In [15]:
blr_linktest(blogit)


Call:
glm(formula = resp ~ fit + fit2, family = binomial(link = "logit"), 
    data = newdat)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.3677  -1.0492  -0.6754   1.1978   2.9171  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.03254    0.04641   0.701    0.483    
fit          0.70526    0.08568   8.231  < 2e-16 ***
fit2        -0.30740    0.06420  -4.789 1.68e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 4279.5  on 3205  degrees of freedom
Residual deviance: 3965.3  on 3203  degrees of freedom
AIC: 3971.3

Number of Fisher Scoring iterations: 5


##### Result of linktest

From the logistic regression output above, both fit and fit2 are significant.  The significance of fit2 indicaets that some important variables have been omitted from the model.  Omitted variables in this sense may mean an interaction term or a linearized variable of some of the predictors.

##### Check interaction terms between Age and female, chronic, white

In [16]:
blogit2 <- glm(ins ~ retire + age + hstatusg + hhincome + educyear + married + hisp + age:female + age:chronic + age:white,
              data=data,family=binomial(link="logit"))

summary(blogit2)


Call:
glm(formula = ins ~ retire + age + hstatusg + hhincome + educyear + 
    married + hisp + age:female + age:chronic + age:white, family = binomial(link = "logit"), 
    data = data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4164  -1.0009  -0.6994   1.2136   2.3762  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.4822944  0.7677232  -1.931 0.053512 .  
retire       0.1727569  0.0855816   2.019 0.043526 *  
age         -0.0189278  0.0115362  -1.641 0.100853    
hstatusg     0.3626005  0.0992086   3.655 0.000257 ***
hhincome     0.0022426  0.0007629   2.940 0.003285 ** 
educyear     0.1147705  0.0143341   8.007 1.18e-15 ***
married      0.5298555  0.0977695   5.419 5.98e-08 ***
hisp        -0.8122282  0.1963483  -4.137 3.52e-05 ***
age:female  -0.0019961  0.0012500  -1.597 0.110302    
age:chronic  0.0005788  0.0004490   1.289 0.197428    
age:white    0.0009366  0.0016072   0.583 0.560056    
---
Signif. codes:  0 '

In [17]:
blr_linktest(blogit2)


Call:
glm(formula = resp ~ fit + fit2, family = binomial(link = "logit"), 
    data = newdat)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.3680  -1.0424  -0.6697   1.1888   2.9293  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.03549    0.04641   0.765    0.444    
fit          0.70612    0.08515   8.292  < 2e-16 ***
fit2        -0.30967    0.06450  -4.801 1.58e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 4279.5  on 3205  degrees of freedom
Residual deviance: 3960.7  on 3203  degrees of freedom
AIC: 3966.7

Number of Fisher Scoring iterations: 5


##### linearize age

In [18]:
blogit3 <- glm(ins ~ retire + age + (age:log(age)) + hstatusg + hhincome + educyear + married + hisp + age:female + age:chronic + age:white,
              data=data,family=binomial(link="logit"))

summary(blogit3)


Call:
glm(formula = ins ~ retire + age + (age:log(age)) + hstatusg + 
    hhincome + educyear + married + hisp + age:female + age:chronic + 
    age:white, family = binomial(link = "logit"), data = data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3877  -1.0051  -0.6938   1.2102   2.3524  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -2.537e+01  1.331e+01  -1.906 0.056693 .  
retire        1.535e-01  8.613e-02   1.782 0.074766 .  
age           1.838e+00  1.033e+00   1.779 0.075260 .  
hstatusg      3.419e-01  9.984e-02   3.425 0.000615 ***
hhincome      2.167e-03  7.604e-04   2.850 0.004377 ** 
educyear      1.147e-01  1.435e-02   7.993 1.32e-15 ***
married       5.384e-01  9.790e-02   5.499 3.82e-08 ***
hisp         -7.995e-01  1.965e-01  -4.069 4.72e-05 ***
age:log(age) -3.566e-01  1.984e-01  -1.797 0.072301 .  
age:female   -2.261e-03  1.259e-03  -1.797 0.072384 .  
age:chronic   6.402e-04  4.505e-04   1.421 0.15532

In [19]:
blr_linktest(blogit3)


Call:
glm(formula = resp ~ fit + fit2, family = binomial(link = "logit"), 
    data = newdat)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.3550  -1.0484  -0.6607   1.1854   2.9000  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.03823    0.04638   0.824     0.41    
fit          0.69393    0.08580   8.087 6.09e-16 ***
fit2        -0.32234    0.06579  -4.899 9.62e-07 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 4279.5  on 3205  degrees of freedom
Residual deviance: 3956.0  on 3203  degrees of freedom
AIC: 3962

Number of Fisher Scoring iterations: 5


Exploring the interaction terms together with the linearization of the age variable, the model still suggest presence of omitted variables since fit2 is statistically significant.  However, given that linktest is a limited tool to detect specification errors, we need to use our best judgement to detect a good model.  This can be done using theoretical propositions.  From our final model, the intraction terms of age with chronic, female and white does not have any significant impact on the dependent variable.

#### Test of Goodness of Fit
* Pseudo-R-square
* Log-likelihood chi-square
* Hosmee and Leseshow
* classification accuracy


#### Goodness of fit with confusion matrix

In [21]:
library(caret)

Warning message:
"package 'caret' was built under R version 4.2.3"
Loading required package: lattice


Attaching package: 'caret'


The following objects are masked from 'package:DescTools':

    MAE, RMSE


The following object is masked from 'package:purrr':

    lift




In [22]:
data$ins <- as.factor(data$ins)

In [23]:
# Split the dataset into train and test set

trainindex <- createDataPartition(data$ins,p = 0.8, list = F)
trainData <- data[trainindex,]
testData <- data[trainindex,]

In [24]:
# train the model

logRegModel <- train(ins ~ .,data = trainData,method ="glm",family ="binomial")

In [25]:
# Use the train model to predict the outcome

predictions <- predict(logRegModel,testData)

In [26]:
# Create confusion matrix

confMatrix <- confusionMatrix(predictions,testData$ins)
print(confMatrix)

Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 1221  547
         1  351  446
                                          
               Accuracy : 0.6499          
                 95% CI : (0.6311, 0.6684)
    No Information Rate : 0.6129          
    P-Value [Acc > NIR] : 5.819e-05       
                                          
                  Kappa : 0.2344          
                                          
 Mcnemar's Test P-Value : 7.655e-11       
                                          
            Sensitivity : 0.7767          
            Specificity : 0.4491          
         Pos Pred Value : 0.6906          
         Neg Pred Value : 0.5596          
             Prevalence : 0.6129          
         Detection Rate : 0.4760          
   Detection Prevalence : 0.6893          
      Balanced Accuracy : 0.6129          
                                          
       'Positive' Class : 0               
                        

The model achieves 65% accuracy by correclty prediction 1221 as having no insurance and 446 as having insurance.  A total of 351 were incorrectly classified as having insurance when they do not hav while 547 wwere classified as having no insurance while they indeed have.

* Pseudo-Rsqure = 6.8%
* Accuracy = 65%
* Sensitivity = 78%
* Specificity = 45%

#### Hosmer and Lemeshow's Goodness of Fit Test

In [27]:
install.packages("glmtoolbox")

Installing package into 'C:/Users/fobem/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)

also installing the dependencies 'RcppGSL', 'RcppZiggurat', 'Rfast', 'statmod'




package 'RcppGSL' successfully unpacked and MD5 sums checked
package 'RcppZiggurat' successfully unpacked and MD5 sums checked
package 'Rfast' successfully unpacked and MD5 sums checked
package 'statmod' successfully unpacked and MD5 sums checked
package 'glmtoolbox' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\fobem\AppData\Local\Temp\Rtmpe0T6JL\downloaded_packages


In [28]:
library(glmtoolbox)

Warning message:
"package 'glmtoolbox' was built under R version 4.2.3"


In [29]:
hltest(lgmodel)


   The Hosmer-Lemeshow goodness-of-fit test

 Group Size Observed  Expected
     1  321       32  42.37315
     2  321       59  74.84060
     3  321       96  92.25235
     4  321      103 106.11623
     5  321      149 121.07176
     6  321      147 134.59408
     7  321      155 145.25786
     8  321      165 154.01920
     9  321      163 172.57265
    10  317      172 197.90212

         Statistic =  32.8312 
degrees of freedom =  8 
           p-value =  6.6054e-05 



The Hosmer and Lemeshow p-value of 0.000066 rejects the null hypothesis of a well-specified model which suggest that our model did not fit the data well

#### Multicollinearity--Misty Package

Two commonly used measures of collinearity are Tolerance and VIF.  If the variables are totally uncorrelated with each other, both the tolrelance and VIF are 1.  If a variable is very closely related to another variable(s), the tolerance value goes to 0 and VIF gets very large.

Going through the collinearity results displayed below, there seem to be no multicollinearity among the variables

In [30]:
library(misty)

Warning message:
"package 'misty' was built under R version 4.2.3"
|-------------------------------------|
| misty 0.5.3 (2023-09-17)            |
| Miscellaneous Functions T. Yanagida |
|-------------------------------------|



In [33]:
## check.collin(lgmodel, print = "all")

In [32]:
check.collin(lgmodel)

 Collinearity Diagnostics

  Tolerance (Tol), Std. Error Inflation Factor (SIF), and Variance Inflation Factor (VIF)

               Estimate Std. Error z value Pr(>|z|)   Tol   SIF   VIF
   (Intercept)   -1.716      0.749  -2.292    0.022                  
   retire         0.197      0.084   2.339    0.019 0.889 1.061 1.125
   age           -0.015      0.011  -1.293    0.196 0.869 1.073 1.150
   hstatusg       0.312      0.092   3.406    0.001 0.897 1.056 1.115
   hhincome       0.002      0.001   3.023    0.003 0.859 1.079 1.164
   educyear       0.114      0.014   8.046    0.000 0.837 1.093 1.195
   married        0.579      0.093   6.201    0.000 0.932 1.036 1.073
   hisp          -0.810      0.196  -4.139    0.000 0.962 1.020 1.040


#### Influential Observations

In [1]:
library(LogisticDx)

ERROR: Error in library(LogisticDx): there is no package called 'LogisticDx'
